# Описание проекта

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 

Предобработка данных не понадобится.


Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.

# Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. 

Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

<b>Инструкция по выполнению проекта</b>

1. Откройте файл с данными и изучите его. Путь к файлу: /datasets/users_behavior.csv. Скачать датасет
2. Разделите исходные данные на обучающую, валидационную и тестовую выборки.
3. Исследуйте качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.
4. Проверьте качество модели на тестовой выборке.
5. Дополнительное задание: проверьте модели на вменяемость. Ничего страшного, если не получится: эти данные сложнее тех, с которыми вы работали раньше. В следующем курсе подробнее об этом расскажем.

## Ход работы:
1. [Загрузка данных](#import)
2. [Разделим исходные данные на обучающую, валидационную и тестовую выборки](#separation)
3. [Исследуем качество разных моделей](#model):
- [Decision Tree](#tree);
- [Random Forest](#forest);
- [Logistic Regression](#logistic);
4. [Проверьте качество модели на тестовой выборке](#test)
5. [Проверка модели на вменяемость](#sanity)
6. [Вывод](#conclusion)

<a id="import"> </a>
# 1. Загрузка данных.

In [48]:
# импорт библиотек

import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [2]:
data = pd.read_csv('users_behavior.csv')

In [3]:
data.head(5)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [6]:
# Проведем подсчет и удаление явных дубликатов

print(data.duplicated().sum())
data = data.drop_duplicates()

0


Размер датасета - (3214, 5), пропущенных значений нет, явных дубликатов - нет.

Целевая переменная - столбец 'is_ultra'.

<a id="separation"> </a>
# 2. Разделим исходные данные на обучающую, валидационную и тестовую выборки.

In [7]:
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']

In [8]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.4, 
                                                                              random_state=12345) 

In [10]:
features_valid, features_test, target_valid, target_test = train_test_split(features_test, target_test, test_size=0.5, 
                                                                              random_state=12345) 

In [11]:
print(f'Исходный размер датасета: {data.shape[0]}')
print(f'Размер обучающей выборки: {features_train.shape[0]}')
print(f'Размер валидационной выборки: {features_valid.shape[0]}')
print(f'Размер тестовой выборки: {features_test.shape[0]}')

Исходный размер датасета: 3214
Размер обучающей выборки: 1928
Размер валидационной выборки: 643
Размер тестовой выборки: 643


Соотношение обучающей, валидационной и тестовой выборки - 3:1:1.

<a id="model"> </a>
# 3. Исследуем качество разных моделей. 

Перед нами стоит задача бинарной классификации.

Для исследования были выбраны модели:
- Decision Tree,
- Random Forest,
- и Logistic Regression.

<a id="tree"> </a> 
# 3.1. Decision Tree

In [12]:
best_dt_model = None
best_dt_params = []
best_dt_result = 0

In [18]:
%%time

for depth in range(1, 10):
    for samples_split in range(2, 10):
        for samples_leaf in range(1, 10):
            for crit in ['gini', 'entropy']:
                dt_tree = DecisionTreeClassifier(random_state=12345, 
                                                 max_depth=depth, 
                                                 criterion=crit,
                                                 min_samples_split=samples_split, 
                                                 min_samples_leaf=samples_leaf)
                dt_tree.fit(features_train, target_train)
                predictions = dt_tree.predict(features_train)
                predictions_valid = dt_tree.predict(features_valid)
                result = dt_tree.score(features_valid, target_valid)

                if result > best_dt_result:
                    best_dt_model = dt_tree
                    best_dt_result = result
                    best_dt_params = [depth, crit, samples_split, samples_leaf]

Wall time: 13.5 s


In [19]:
print(f'Accuracy наилучшей модели на валидационной выборке: {best_dt_result}',
      f'max_depth: {best_dt_params[0]}',
      f'criterion: {best_dt_params[1]}',
      f'min_samples_split: {best_dt_params[2]}',
      f'min_samples_leaf:  {best_dt_params[3]}', sep='\n')  

Accuracy наилучшей модели на валидационной выборке: 0.7900466562986003
max_depth: 6
criterion: entropy
min_samples_split: 2
min_samples_leaf:  4


<a id="forest"> </a>
# 3.2. Random Forest

In [14]:
best_rf_model = None
best_rf_result = 0
best_rf_params = []

In [22]:
%%time

for est in range(1, 10):
    for depth in range(1, 10):
        for samples_split in range(2, 10):
            for samples_leaf in range(1, 10):
                for crit in ['gini', 'entropy']:
                    r_forest = RandomForestClassifier(random_state=12345,
                                                      n_estimators=est,
                                                      max_depth=depth,
                                                      criterion=crit,
                                                      min_samples_split=samples_split,
                                                      min_samples_leaf=samples_leaf)
                    r_forest.fit(features_train, target_train)
                    predictions = r_forest.predict(features_train)
                    predictions_valid = r_forest.predict(features_valid)
                    result = r_forest.score(features_valid, target_valid)
    
                    if result > best_rf_result:
                        best_rf_model = r_forest
                        best_rf_result = result
                        best_rf_params = [est, depth, crit, samples_split, samples_leaf]

Wall time: 4min 14s


In [23]:
print(f'Accuracy наилучшей модели на валидационной выборке: {best_rf_result}',
      f'n_estimators: {best_rf_params[0]}',
      f'max_depth: {best_rf_params[1]}',
      f'criterion: {best_rf_params[2]}',
      f'min_samples_split: {best_rf_params[3]}',
      f'min_samples_leaf:  {best_rf_params[4]}', sep='\n')  

Accuracy наилучшей модели на валидационной выборке: 0.8118195956454122
n_estimators: 8
max_depth: 10
criterion: gini
min_samples_split: 5
min_samples_leaf:  2


<a id="logistic"> </a>
# 3.3. Logistic Regression

In [42]:
%%time

lr_model = LogisticRegression(random_state=12345, solver='liblinear', max_iter=100) 
lr_model.fit(features_train, target_train)
lr_predictions = lr_model.predict(features_train)
lr_predictions_valid = lr_model.predict(features_valid)

lr_result = lr_model.score(features_valid, target_valid)
print(f'Accuracy модели на валидационной выборке: {lr_result}')

Accuracy модели на валидационной выборке: 0.7573872472783826
Wall time: 19.9 ms


Наилучшее значение accuracy показала модель RandomForest. Главный минус модели - продолжительность обучения.

<a id="test"> </a>
# 4. Проверьте качество модели на тестовой выборке.

Лучшие результаты показаласа модель RandomForestClassifier с параметрами:
- n_estimators=8, 
- max_depth=10, 
- criterion='gini',
- min_samples_split=5,
- min_samples_leaf=2.


Точность составила 0.8118195956454122.

Обучим модель с этими параметрами на тестовой выборке.

In [29]:
model = RandomForestClassifier(random_state=12345, n_estimators=8, max_depth=10, criterion='gini',
                          min_samples_split=5, min_samples_leaf=2)

model.fit(features_train, target_train)
model_predictions = model.predict(features_test)
model_accuracy = accuracy_score(target_test, model_predictions)

print(f'Точность модели на тестовой выборке: {model_accuracy}')

Точность модели на тестовой выборке: 0.7978227060653188


Точность на валидационной выборке и тестовой практически совпадают - 81% и 79.8%. 

<a id="sanity"> </a>
# 5. Проверка модели на вменяемость

In [30]:
target.value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

Имеющийся признак несбалансирован - объектов класса "0" в 2.3 раза больше, чем объектов класса "1".

In [60]:
# Создаим константную модель: любому объекту она прогнозирует класс «0»

const = np.zeros(target_test.shape[0])
target_pred_constant = pd.Series(const)

In [61]:
accuracy_score(target_test, target_pred_constant)

0.6842923794712286

Accuracy константной модели - 68%.

Модель проверку на вменяемость прошла (?)

<a id="conclusion"> </a>
# 6. Вывод.

Размер датасета - (3214, 5), пропущенных значений нет, явных дубликатов - нет. Целевая переменная - столбец 'is_ultra'.

Мы разделили имеющийся датасет на обучающую, валидационную и тестовую выборки в соотношении 3:1:1. В обучающую выборку вошло 1928 объектов, в валидационную и тестовую - по 643 объекта.

Перед нами стояла задача бинарной классификации.
Для исследования были выбраны модели:
- Decision Tree,
- Random Forest,
- и Logistic Regression.

Наилучшее значение accuracy показала модель Random Forest (81%), ее мы будем в дальнейшем использовать. Главный минус - временные затраты: обучение модели и подбор параметров заняли 4 минуты 20 секунд, обучение остальных моделей - 15-20 секунд.

Модели Decision Tree и Logistic Regression показали точность в 79% и 75%, соответственно.

Мы проверили качество модели на тестовой выборке - точность на валидационной выборке и тестовой практически совпадают - 81% и 79.8%.